# Grupo 4 ->Nuno Mendes (2727), Rosário Silva (21138), Tiago Azevedo (21153), Francisco Pereira (21156)

## Introdução

Este trabalho tem como objetivo desenvolver um agente inteligente, documentando as etapas da criação do projeto e respetiva implementação. 
O tema que o nosso grupo escolheu foi o Horário das aulas. Tema este que foi proposto pelo professor Joaquim Silva, através do enunciado disponibilizado para este trabalho prático.
O exercício consiste na formulação de uma solução de um horário escolar para várias turmas, com algumas restrições predefinidas, que serão abordadas de seguida. O principal objetivo será minimizar o número de dias que cada aluno tem de viajar para o campus e, por consequência, reduzir o número de salas utilizadas.


Para a realização deste agente, utilizamos 2 abordagens distintas que estarão explicadas a baixo em detalhe. 

## Link Git: 

    https://github.com/tiagoazevedo22/LESI3_SmartCampus.git

## Abordagem 1:

In [3]:
Disciplinas = 'Portugues Ingles Matematica Historia Informatica'.split()
Dias = 'Segunda Terca Quarta Quinta Sexta '.split()
Horas = 'H8 H10 H12 H14 H16'.split()
Tipo = 'Online Presencial'.split()
Turma = 'T1 T2 T3 T4 T5 '.split()
Sala = 'S1 S2 S3 S4 S5'.split()
variaveis = list(Disciplinas + Dias + Horas + Tipo + Turma + Sala)

dominio = {}
for var in variaveis:
    dominio[var] = list(range(1,6))

# constraints
restricoes = [
              Constraint(Disciplinas, all_diff_constraint),
              Constraint(Horas, all_diff_constraint),
              Constraint(Dias, all_diff_constraint),
              Constraint(Turma, all_diff_constraint),
              Constraint(Tipo, all_diff_constraint),
              Constraint(Sala, all_diff_constraint),
              Constraint(('Segunda','Ingles'), eq),
            
              ]

# days_too_short
days_too_short = NaryCSP(dominio, restricoes)
print(days_too_short.variables)

# Result
k = ac_solver(days_too_short, arc_heuristic=sat_up)

#Print result
for h in range(1,6):
    print('Solucao', h, end=' ')
    for (var,val) in k.items():
        if val == h:
            print(var, end=' ')
    print()

{'T2', 'S5', 'Ingles', 'T1', 'T5', 'Sexta', 'S4', 'Matematica', 'H12', 'T4', 'Segunda', 'H8', 'H10', 'Terca', 'Presencial', 'H14', 'S2', 'Quinta', 'T3', 'Online', 'S1', 'Portugues', 'Quarta', 'S3', 'H16', 'Informatica', 'Historia'}
Solucao 1 Ingles Segunda H12 Presencial T2 S5 
Solucao 2 Matematica Sexta H8 Online T1 S4 
Solucao 3 Portugues Terca H10 T5 S2 
Solucao 4 Informatica Quinta H14 T4 S1 
Solucao 5 Historia Quarta H16 T3 S3 


## Abordagem 2:

### Código completo com execução:

In [4]:
dominio = {
    'name1': set(['lesson1']), 'dia1' : set(range(2,6+1)), 'dur1' : set(range(1,2+1)), 'hora1' : set(range(8,17+1)), 'tipo1' : set(['online','presencial']),
    'name2': set(['lesson2']), 'dia2' : set(range(2,6+1)), 'dur2' : set(range(1,2+1)), 'hora2' : set(range(8,17+1)), 'tipo2' : set(['online','presencial']),
    'name3': set(['lesson3']), 'dia3' : set(range(2,6+1)), 'dur3' : set(range(1,2+1)), 'hora3' : set(range(8,17+1)), 'tipo3' : set(['online','presencial']),
    #'name4': set(['lesson7']), 'dia7' : set(range(2,6+1)), 'dur7' : set(range(1,2+1)), 'hora7' : set(range(8,17+1)), 'tipo7' : set(['online','presencial']),
    #'name5': set(['lesson8']), 'dia8' : set(range(2,6+1)), 'dur8' : set(range(1,2+1)), 'hora8' : set(range(8,17+1)), 'tipo8' : set(['online','presencial']),
    #'name6': set(['lesson9']), 'dia9' : set(range(2,6+1)), 'dur9' : set(range(1,2+1)), 'hora9' : set(range(8,17+1)), 'tipo9' : set(['online','presencial']),
    #'name7': set(['lesson10']), 'dia10' : set(range(2,6+1)), 'dur10' : set(range(1,2+1)), 'hora10' : set(range(8,17+1)), 'tipo10' : set(['online','presencial']),
}        

class lesson:
    
    def __init__(self, name ,dia, dur, hora, tipo):
        self.name = name
        self.dia = dia
        self.dur = dur
        self.hora = hora
        self.tipo = tipo

    @staticmethod
    def cond1_all_lessons_friday_durantion2hours(l):
        for x  in l:
            if x.dia == 4 and x.dur != 2:
                return False
        return True
    
    @staticmethod
    def cond2_one_or_two_are_online(l):
        online = 0
        for x  in l:
            if x.tipo == 'online':
                online = online + 1
        if online > 0 and online <= 2:
            return True
        return False
    
    @staticmethod
    def cond3_class_should_not_have_more_3_lessons_day(l):
        lessons_per_day = [0,0,0,0,0]
        for x  in l:
            lessons_per_day[x.dia-2] = lessons_per_day[x.dia-2] + 1
        
        for ld in lessons_per_day:
            if ld > 3:
                return False
        return True
    
    @staticmethod
    def cond4_online_lessons_cannot_be_booked_immediately(l):
        return True
    
    @staticmethod
    def cond5_only_2_lessons_per_half_day(l):
        lessons_per_morning = [0,0,0,0,0]
        lessons_per_afternoon = [0,0,0,0,0]
        for x  in l:
            if x.hora <= 13:
                lessons_per_morning[x.dia-2] = lessons_per_morning[x.dia-2] + 1
            else:
                lessons_per_afternoon[x.dia-2] = lessons_per_afternoon[x.dia-2] + 1

        for ld in lessons_per_morning:
            if ld > 2:
                return False
        
        for ld in lessons_per_afternoon:
            if ld > 2:
                return False
            
        return True
    
    @staticmethod
    def cond6_two_classes_specific_classRoom(l):
        return True
    
    @staticmethod
    def cond(l):
        if lesson.cond1_all_lessons_friday_durantion2hours(l) == False:
            return False
        if lesson.cond2_one_or_two_are_online(l) == False:
            return False
        if lesson.cond3_class_should_not_have_more_3_lessons_day(l) == False:
            return False
        if lesson.cond4_online_lessons_cannot_be_booked_immediately(l) == False:
            return False
        if lesson.cond5_only_2_lessons_per_half_day(l) == False:
            return False
        if lesson.cond6_two_classes_specific_classRoom(l) == False:
            return False
        return True
        
    
def teste_condicoes(*values):
    i = 0
    lessons = []
    while i < len(values):
        lessons.append(lesson(values[i], values[i+1], values[i+2], values[i+3], values[i+4]))
        i += 5   
        
    #print(values)
    
    return lesson.cond(lessons)

# constraints
restricoes = [Constraint(tuple(dominio.keys()), teste_condicoes),]

# days_too_short
days_too_short = NaryCSP(dominio, restricoes)
print(days_too_short.variables)


# Result
k = ac_solver(days_too_short, arc_heuristic=sat_up)
print(k)

{'tipo1', 'dur2', 'name3', 'tipo3', 'dur3', 'dur1', 'name2', 'dia2', 'dia3', 'dia1', 'tipo2', 'hora2', 'name1', 'hora1', 'hora3'}
{'name1': 'lesson1', 'dia1': 2, 'dur1': 1, 'hora1': 8, 'tipo1': 'online', 'name2': 'lesson2', 'dia2': 2, 'dur2': 1, 'hora2': 8, 'tipo2': 'presencial', 'name3': 'lesson3', 'dia3': 2, 'dur3': 1, 'hora3': 16, 'tipo3': 'online'}


### Código explicado:

O dominio foi criado com o intuito de introduzir valores válidos/possíveis para serem executados pelo agente.
Neste caso:
- Cada aula pode ser de Segunda a Sexta;
- tem a duração de 1 ou 2 horas;
- as aulas decorrem entre as 8h e as 17h;
- as aulas podem ser online ou presencial.

In [ ]:
dominio = {
    'name1': set(['lesson1']), 'dia1' : set(range(2,6+1)), 'dur1' : set(range(1,2+1)), 'hora1' : set(range(8,17+1)), 'tipo1' : set(['online','presencial']),
    'name2': set(['lesson2']), 'dia2' : set(range(2,6+1)), 'dur2' : set(range(1,2+1)), 'hora2' : set(range(8,17+1)), 'tipo2' : set(['online','presencial']),
    'name3': set(['lesson3']), 'dia3' : set(range(2,6+1)), 'dur3' : set(range(1,2+1)), 'hora3' : set(range(8,17+1)), 'tipo3' : set(['online','presencial']),
    #'name4': set(['lesson7']), 'dia7' : set(range(2,6+1)), 'dur7' : set(range(1,2+1)), 'hora7' : set(range(8,17+1)), 'tipo7' : set(['online','presencial']),
    #'name5': set(['lesson8']), 'dia8' : set(range(2,6+1)), 'dur8' : set(range(1,2+1)), 'hora8' : set(range(8,17+1)), 'tipo8' : set(['online','presencial']),
    #'name6': set(['lesson9']), 'dia9' : set(range(2,6+1)), 'dur9' : set(range(1,2+1)), 'hora9' : set(range(8,17+1)), 'tipo9' : set(['online','presencial']),
    #'name7': set(['lesson10']), 'dia10' : set(range(2,6+1)), 'dur10' : set(range(1,2+1)), 'hora10' : set(range(8,17+1)), 'tipo10' : set(['online','presencial']),
}       

Criou-se esta classe para receber os dados a serem tratados pelo agente e, posteriormente, passarem pelas diversas restrições, sendo elas:
- cond1_all_lessons_friday_durantion2hours(l):
    - Representa todas as aulas da quarta têm de ter duração de 2 horas;
- cond2_one_or_two_are_online(l):
    - 1 ou 2 aulas têm de ser online;
- cond3_class_should_not_have_more_3_lessons_day(l):
    - Cada turma não pode de ter mais 3 aulas por dia;
- cond4_online_lessons_cannot_be_booked_immediately(l):
    - As aulas online não podem ser reservadas após uma aula presencial;
- cond5_only_2_lessons_per_half_day(l):
    - Só pode haver 2 aulas de manhã ou de tarde;
- cond6_two_classes_specific_classRoom(l):
    - 2 turmas em salas específicas;

Após a declaração das diferentes restrições, foi criado teste_condicoes() com o intuito de verificar as restrições efetuadas anteriormente e com base nestas guardar todos os valores num array para ser mostrado no resultado final. 

In [ ]:
class lesson:
    
    def __init__(self, name ,dia, dur, hora, tipo):
        self.name = name
        self.dia = dia
        self.dur = dur
        self.hora = hora
        self.tipo = tipo

    @staticmethod
    def cond1_all_lessons_friday_durantion2hours(l):
        for x  in l:
            if x.dia == 4 and x.dur != 2:
                return False
        return True
    
    @staticmethod
    def cond2_one_or_two_are_online(l):
        online = 0
        for x  in l:
            if x.tipo == 'online':
                online = online + 1
        if online > 0 and online <= 2:
            return True
        return False
    
    @staticmethod
    def cond3_class_should_not_have_more_3_lessons_day(l):
        lessons_per_day = [0,0,0,0,0]
        for x  in l:
            lessons_per_day[x.dia-2] = lessons_per_day[x.dia-2] + 1
        
        for ld in lessons_per_day:
            if ld > 3:
                return False
        return True
    
    @staticmethod
    def cond4_online_lessons_cannot_be_booked_immediately(l):
        return True
    
    @staticmethod
    def cond5_only_2_lessons_per_half_day(l):
        lessons_per_morning = [0,0,0,0,0]
        lessons_per_afternoon = [0,0,0,0,0]
        for x  in l:
            if x.hora <= 13:
                lessons_per_morning[x.dia-2] = lessons_per_morning[x.dia-2] + 1
            else:
                lessons_per_afternoon[x.dia-2] = lessons_per_afternoon[x.dia-2] + 1

        for ld in lessons_per_morning:
            if ld > 2:
                return False
        
        for ld in lessons_per_afternoon:
            if ld > 2:
                return False
            
        return True
    
    @staticmethod
    def cond6_two_classes_specific_classRoom(l):
        return True
    
    @staticmethod
    def cond(l):
        if lesson.cond1_all_lessons_friday_durantion2hours(l) == False:
            return False
        if lesson.cond2_one_or_two_are_online(l) == False:
            return False
        if lesson.cond3_class_should_not_have_more_3_lessons_day(l) == False:
            return False
        if lesson.cond4_online_lessons_cannot_be_booked_immediately(l) == False:
            return False
        if lesson.cond5_only_2_lessons_per_half_day(l) == False:
            return False
        if lesson.cond6_two_classes_specific_classRoom(l) == False:
            return False
        return True
        
    
def teste_condicoes(*values):
    i = 0
    lessons = []
    while i < len(values):
        lessons.append(lesson(values[i], values[i+1], values[i+2], values[i+3], values[i+4]))
        i += 5   
        
    #print(values)
    
    return lesson.cond(lessons)